In [1]:
!pip install ultralytics
!pip install -U ipywidgets

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/33/b1/f2cc216c80bee008db680d78026ee1a0c5421755f4894c8ec27dc773fbe0/ultralytics-8.1.4-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.5/701.5 kB 13.5 MB/s eta 0:00:00
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0

In [2]:
!mkdir -p /kaggle/temp/train/labels
!mkdir -p /kaggle/temp/train/images

!mkdir -p /kaggle/temp/val/labels
!mkdir -p /kaggle/temp/val/images

In [3]:
!cp /kaggle/input/kidney-1-dense-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-dense-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-1-voi-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-voi-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-2-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-2-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-3-sparse-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-3-sparse-to-yolo/train/images/* /kaggle/temp/train/images

In [4]:
!cp /kaggle/input/kidney-3-dense-to-yolo/train/labels/* /kaggle/temp/val/labels
!cp /kaggle/input/kidney-3-dense-to-yolo/train/images/* /kaggle/temp/val/images

In [5]:
# !tail /kaggle/input/kidney-1-voi-to-yolo/train/kidney_1_voi.json

# Write yolo dataset yaml

In [6]:
import yaml

In [7]:
!mkdir train

In [8]:
train_path = '/kaggle/temp/train'
val_path = '/kaggle/temp/val'
names = {0: 'Vessel'}
yaml_path = 'train/train.yaml'

# Number of classes
nc = len(names)

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': nc,
    'test': '',
    'train': train_path,
    'val': val_path
}

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [9]:
!cat {yaml_path}

names:
  0: Vessel
nc: 1
test: ''
train: /kaggle/temp/train
val: /kaggle/temp/val


In [10]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")
wandb.login(key=wandb_api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
from ultralytics import YOLO

model = YOLO('yolov8m-seg.pt')

100%|██████████| 52.4M/52.4M [00:00<00:00, 212MB/s]


In [12]:
project = 'hacking_human_vasculature'
name = 'kidney_3_dense_val_rest_train_yolo_m_1024_9'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=100,
                      time=5,
                      patience=10, 
                      batch=9,
                      imgsz=1024,
                      cache=True
                     )

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=train/train.yaml, epochs=100, time=5, patience=10, batch=9, imgsz=1024, save=True, save_period=-1, cache=True, device=None, workers=8, project=hacking_human_vasculature, name=kidney_3_dense_val_rest_train_yolo_m_1024_9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

wandb: Currently logged in as: aaalex-lit. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240122_175918-2uan6ead
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run kidney_3_dense_val_rest_train_yolo_m_1024_9
wandb: ⭐️ View project at https://wandb.ai/aaalex-lit/hacking_human_vasculature
wandb: 🚀 View run at https://wandb.ai/aaalex-lit/hacking_human_vasculature/runs/2uan6ead


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

                   all       1035      41540      0.526      0.353      0.316      0.207      0.389      0.266      0.232      0.116



18 epochs completed in 5.014 hours.
Optimizer stripped from hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_m_1024_9/weights/last.pt, 54.9MB
Optimizer stripped from hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_m_1024_9/weights/best.pt, 54.9MB

Validating hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_m_1024_9/weights/best.pt...
Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8m-seg summary (fused): 245 layers, 27222963 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [01:31<00:00,  1.59s/it]


                   all       1035      41540      0.526      0.354      0.317      0.207      0.389      0.267      0.232      0.116
Speed: 1.1ms preprocess, 37.3ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_m_1024_9


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆██▇▇▆▆▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg1 ▃▆██▇▇▆▆▅▅▄▄▃▃▂▂▁▁
wandb:                  lr/pg2 ▃▆██▇▇▆▆▅▅▄▄▃▃▂▂▁▁
wandb:        metrics/mAP50(B) ▁▃▃▂▄▄▅▄▇▇▆▆▇▇▇▇▇█
wandb:        metrics/mAP50(M) ▁▃▅▂▆▁▆▅▇▇▅▇▆▆▆▇▆█
wandb:     metrics/mAP50-95(B) ▁▂▂▁▃▃▄▃▆▇▅▆▇▇▇▇▇█
wandb:     metrics/mAP50-95(M) ▂▃▅▂▆▁▆▆▇▇▅▇▆▆▇█▆█
wandb:    metrics/precision(B) ▁▄▃▄▄▅▆▅▇▇▇▆█▇▇███
wandb:    metrics/precision(M) ▄▄▄▄█▁▆██▆▇▇▇▇▇▇▇█
wandb:       metrics/recall(B) ▁▂▄▂▄▅▅▆▇▇▇▇█▇████
wandb:       metrics/recall(M) ▃▂▇▃▅▁▄█▃▃▄▆▃▃▄▅▃▄
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▇█▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁
wandb:          train/cls_loss █▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
wandb:          train/dfl_loss █▄▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁
wandb:          train/seg_loss █▄▄▃▃▂▂▃▂▂▂▂▂▁▁▁▁▁
wandb:            val/box_loss ▇▇▇█▇

In [13]:
wandb.finish()